In [5]:
import numpy as np
import pandas as pd
import requests
import io
from scipy import optimize
import matplotlib.pyplot as plt
import requests, pandas as pd
from bs4 import BeautifulSoup
import urllib.request

In [2]:
url='https://en.wikipedia.org/wiki/List_of_urban_areas_in_the_United_Kingdom#cite_note-6'

In [3]:
tables=pd.read_html('https://en.wikipedia.org/wiki/List_of_urban_areas_in_the_United_Kingdom#cite_note-6')

In [10]:
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")


In [59]:
all_tables=soup.find_all("table")

right_table=soup.find_all("table", class_="wikitable sortable")
actual_table=soup.find_all("tbody")
actual_table[0]

<tbody><tr>
<th>#
</th>
<th>Area<sup class="reference" id="cite_ref-BUA_4-0"><a href="#cite_note-BUA-4">[4]</a></sup><sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[note 1]</a></sup>
</th>
<th>Population<br/>(2011)
</th>
<th>Area (km<sup>2</sup>)<sup class="reference" id="cite_ref-6"><a href="#cite_note-6">[note 2]</a></sup>
</th>
<th>Density (People/km<sup>2</sup>)
</th>
<th class="unsortable">Primary sub-divisions<sup class="reference" id="cite_ref-7"><a href="#cite_note-7">[note 3]</a></sup>
</th>
<th>Notable changes from 2001 to 2011<sup class="reference" id="cite_ref-8"><a href="#cite_note-8">[5]</a></sup><sup class="reference" id="cite_ref-9"><a href="#cite_note-9">[note 4]</a></sup>
</th></tr>
<tr>
<th>1
</th>
<td><span data-sort-value="London" style="display:none;"></span><a href="/wiki/Greater_London_Built-up_Area" title="Greater London Built-up Area">Greater London</a>
</td>
<td>9,787,426
</td>
<td>1,737.9
</td>
<td>5,630
</td>
<td><a href="/wiki/London_borough

In [74]:
table=soup.find('tbody')
headings=table.findAll('th',text="Area")
headings_list=[i.text for i in headings]
headings_list

[]

In [71]:
values=table.findAll('td')

In [73]:
headings_list

[]

In [79]:
Area=[]
Population=[]
Area_size=[]
Density=[]
Primary_sub_division=[]
Notable_changes_from_2001_to_2011=[]

for row in actual_table[0].findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==6:
        Area.append(cells[0].find(text=True))
        Population.append(cells[1].find(text=True))
        Area_size.append(cells[2].find(text=True))
        Density.append(cells[3].find(text=True))
        Primary_sub_division.append(cells[4].find(text=True))
        Notable_changes_from_2001_to_2011.append(cells[5].find(text=True))

In [80]:
df=pd.DataFrame(Area,columns=['Area'])
df['Population']=Population
df['Area_size']=Area_size
df['Density']=Density
df['Primary_sub_division']=Primary_sub_division
df['Notable_changes_from_2001_to_2011']=Notable_changes_from_2001_to_2011

In [96]:
df['Area_size']=df['Area_size'].str.replace('\n','')
Area_size_data = pd.DataFrame(df['Area_size'])
Area_size_data['City'] = df.Area


,Area_size,City
0,"1,737.9",Greater London
1,630.3,Greater Manchester
2,598.9,West Midlands
3,487.8,West Yorkshire
4,368.5,Greater Glasgow
...,...,...
71,26.2,Chelmsford
72,29.4,Basingstoke
73,25.4,Maidstone
74,24.8,Bedford


In [98]:
data = 'https://raw.githubusercontent.com/107SBakst/107SBakst.github.io/main/data/Centre_for_cities_full_data.csv'


In [118]:
dataset = pd.read_csv(data, encoding= 'unicode_escape',error_bad_lines=False)
df = pd.DataFrame(dataset)
df = df[:62]
df.columns = df.columns.str.replace('ï»¿', '')
df.columns = df.columns.str.replace('(ÃÂ£)', '')

In [121]:
df_right_merge = pd.merge(Area_size_data, df, how='right')


In [123]:
df_right_merge.to_csv(r'C:\Users\vw191\OneDrive - University of Bristol\Course\3rd year\Data Science\Project\Chart_2_UK\Chart2_project_full_dataset.csv')